In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import date

# Data Loading

SF DBI’s development pipeline datasets use four nearly-identical data models. Parcel identificator was called “BLKLOT” or “Block Lot” in older datasets and is called APN (Assessor's Parcel Number) in the most recent datasets. The same parcel id can written with a leading zero in one dataset and without a leading zero in another one. Prior to 2014 Q3, datasets don’t have fields for affordable unit counts (total affordable units and net affordable units).

Some project records don’t include building permit id or use the placeholder “MULTIPLE” instead of actual permit references. Initially, we remove permitless projects from the dataset, than re-add them at a later stage.

In [17]:
def loadData(fileName, label, fmt = 1):
    print(fileName)
    columns = ['UNITS', 'NET_UNITS', 'AFF_UNITS', 'NET_AFF_UNITS', 'NAMEADDR', 'APN', 'BESTSTAT', 'BESTDATE', 'LOCATION']
    
    converters = { 'APN': lambda x: x[4:], 'BP_APPLNO': lambda x: 'N'+x.replace(',','') }
    if fmt == 2 or fmt == 3:
        converters = { 'BLKLOT': lambda x: x.lstrip('0'), 'BP_APPLNO': lambda x: 'N'+x.replace(',','') }
    if fmt == 4:

        converters = { 'Block Lot': lambda x: x.lstrip('0'), 'DBI Permit': lambda x: 'N'+x.replace(',','')}
        
    X = pd.read_csv(fileName, sep=',', parse_dates=[], infer_datetime_format=True, quotechar='"', converters=converters)
    
    if fmt == 2 or fmt == 3:
        X = X.rename(columns={"AFF_UNITS_NET": "NET_AFF_UNITS", "BLKLOT": "APN"})
    if fmt == 3:
        X = X.rename(columns={"UNITSNET": "NET_UNITS", "Geography": "LOCATION"})
    if fmt == 4:
        X['LOCATION'] = X['Location 1']#.apply(lambda x: x.split('\n')[1])
        X['NAMEADDR'] = X['Location 1'].apply(lambda x: x.split('\n')[0])
        
        X = X.rename(columns={"Units": "UNITS", "Net Added Units": "NET_UNITS", "Best Stat": "BESTSTAT", "Best Date": "BESTDATE", 'Block Lot': 'APN',
                             'DBI Permit': 'BP_APPLNO'})
        X['NET_AFF_UNITS'] = 0
        X['AFF_UNITS'] = 0
    if fmt == 5:
        X = X.rename(columns={"PROPUSE": "PROJECT_TYPE"})
    X = X.rename(columns={"Location": "LOCATION"})
    X['LOCATION'] = X['LOCATION'].apply(lambda x: str(x).split('\n')[-1])
    #X = X[X['BESTSTAT'] == 'CONSTRUCTION']
    X = X[X['UNITS'] > 0]
    X = X[X['NET_UNITS'] >= 0]
    
    # Filtering records without permits
    M = X[(X['BP_APPLNO'] == 'N') | (X['BP_APPLNO'] == 'NMULTIPLE')][columns+['BP_APPLNO']]
    X = X[(X['BP_APPLNO'] != 'N') & (X['BP_APPLNO'] != 'NMULTIPLE')]
    
    # Filtering duplicated records
    D = X[X.duplicated('BP_APPLNO', keep=False)][columns+['BP_APPLNO']].sort_values('BP_APPLNO')
    X=X[~X.duplicated('BP_APPLNO')]
    
    # Set unique index by Permit ID
    X = X.set_index('BP_APPLNO')
    X.index.names = ['PERMIT_ID']
    X['BESTDATE'] = pd.to_datetime(X["BESTDATE"])
    X[['UNITS', 'NET_UNITS', 'NET_AFF_UNITS', 'AFF_UNITS']] = X[['UNITS', 'NET_UNITS', 'NET_AFF_UNITS', 'AFF_UNITS']].astype(int)
    
    X = X[columns]
    
    return X,M,D

# Helper Indexes

y13q4 = 0
y14q1 = 1
y14q2 = 2
y14q3 = 3
y14q4 = 4
y15q1 = 5
y15q2 = 6
y15q3 = 7
y15q4 = 8
y16q1 = 9
y16q2 = 10
y16q3 = 11
y16q4 = 12
y17q1 = 13

# Files and versions

files = [
    {'label': '2013\'Q4', 'file': 'data/San_Francisco_Development_Pipeline_2013_Quarter_4.csv', 'format': 4},
    {'label': '2014\'Q1', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_1.csv', 'format': 4},
    {'label': '2014\'Q2', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_2.csv', 'format': 4},
    {'label': '2014\'Q3', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_3.csv', 'format': 4},
    {'label': '2014\'Q4', 'file': 'data/San_Francisco_Development_Pipeline_2014_Quarter_4.csv', 'format': 3},
    {'label': '2015\'Q1', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_1.csv', 'format': 2},
    {'label': '2015\'Q2', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_2.csv', 'format': 1},
    {'label': '2015\'Q3', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_3.csv', 'format': 1},
    {'label': '2015\'Q4', 'file': 'data/San_Francisco_Development_Pipeline_2015_Quarter_4.csv', 'format': 1},
    {'label': '2016\'Q1', 'file': 'data/SF_Development_Pipeline_2016_Q1.csv', 'format': 1},
    {'label': '2016\'Q2', 'file': 'data/SF_Development_Pipeline_2016_Q2.csv', 'format': 5},
    {'label': '2016\'Q3', 'file': 'data/SF_Development_Pipeline_2016_Q3.csv', 'format': 1},
    {'label': '2016\'Q4', 'file': 'data/SF_Development_Pipeline_2016_Q4.csv', 'format': 1},
    {'label': '2017\'Q1', 'file': 'data/SF_Development_Pipeline_2017_Q1.csv', 'format': 1}
]

# Loading Data
count = len(files)
labels = []
data = []
missing = []
duplicates = []

for f in files:
    t,m,d = loadData(f['file'], f['label'], f['format'])
    data.append(t)
    missing.append(m)
    duplicates.append(d)
    labels.append(f['label'])

data/San_Francisco_Development_Pipeline_2013_Quarter_4.csv
data/San_Francisco_Development_Pipeline_2014_Quarter_1.csv
data/San_Francisco_Development_Pipeline_2014_Quarter_2.csv
data/San_Francisco_Development_Pipeline_2014_Quarter_3.csv
data/San_Francisco_Development_Pipeline_2014_Quarter_4.csv
data/San_Francisco_Development_Pipeline_2015_Quarter_1.csv
data/San_Francisco_Development_Pipeline_2015_Quarter_2.csv
data/San_Francisco_Development_Pipeline_2015_Quarter_3.csv
data/San_Francisco_Development_Pipeline_2015_Quarter_4.csv
data/SF_Development_Pipeline_2016_Q1.csv
data/SF_Development_Pipeline_2016_Q2.csv
data/SF_Development_Pipeline_2016_Q3.csv
data/SF_Development_Pipeline_2016_Q4.csv
data/SF_Development_Pipeline_2017_Q1.csv


# Displaying duplicated records
Printing out all records with duplicate building permit id for manual inspection.

In [5]:
for i in range(count):
    if (len(duplicates[i]) > 0):
        print()
        print("Duplicated records at {}".format(labels[i]))
        display(duplicates[i])


Duplicated records at 2013'Q4


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE,LOCATION,BP_APPLNO
265,7.0,7.0,0,0,"(37.740113999999998, -122.389089)",5260002,BP ISSUED,08-Aug-13,4132 Third Street,N200707055953
812,7.0,7.0,0,0,"(37.740068999999998, -122.388912)",5260003,BP ISSUED,08-Aug-13,4132 03rd St,N200707055953
188,2.0,1.0,0,0,"(37.754974400000002, -122.4076295)",4152016,CONSTRUCTION,21-Feb-12,1076 Hampshire St,N200709193092
515,2.0,2.0,0,0,"(37.754857000000001, -122.407522)",4152046,CONSTRUCTION,21-Feb-12,1078 Hampshire St,N200709193092
127,1.0,1.0,0,0,"(37.747147900000002, -122.4509959)",2821010,BP APPROVED,02-Mar-12,83 Panorama Dr,N200711077576
527,1.0,1.0,0,0,"(37.747062999999997, -122.45041500000001)",2821023,BP APPROVED,02-Mar-12,83 Panorama Dr,N200711077576
299,2.0,1.0,0,0,"(37.725757999999999, -122.42266100000001)",5943008,CONSTRUCTION,13-Sep-13,268 Madison St,N200711077587
520,1.0,1.0,0,0,"(37.725698000000001, -122.422527)",5943051,CONSTRUCTION,13-Sep-13,268 Madison St,N200711077587
307,1.0,1.0,0,0,"(37.714095, -122.404408)",6208003,BP ISSUED,25-Apr-13,138 Alpha St,N200806194898
530,1.0,1.0,0,0,"(37.714331000000001, -122.404082)",6208056,BP ISSUED,25-Apr-13,138 Alpha St,N200806194898



Duplicated records at 2014'Q1


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE,LOCATION,BP_APPLNO
657,7.0,7.0,0,0,"(37.740068999999998, -122.388912)",5260003,CONSTRUCTION,26-Feb-14,4132 03rd St,N200707055953
124,7.0,7.0,0,0,"(37.740113999999998, -122.389089)",5260002,CONSTRUCTION,26-Feb-14,4132 Third Street,N200707055953
60,2.0,1.0,0,0,"(37.754974400000002, -122.4076295)",4152016,CONSTRUCTION,21-Feb-12,1076 Hampshire St,N200709193092
627,2.0,2.0,0,0,"(37.754857000000001, -122.407522)",4152046,CONSTRUCTION,21-Feb-12,1078 Hampshire St,N200709193092
756,1.0,1.0,0,0,"(37.747062999999997, -122.45041500000001)",2821023,BP APPROVED,02-Mar-12,83 Panorama Dr,N200711077576
281,1.0,1.0,0,0,"(37.747147900000002, -122.4509959)",2821010,BP APPROVED,02-Mar-12,83 Panorama Dr,N200711077576
748,1.0,1.0,0,0,"(37.725698000000001, -122.422527)",5943051,CONSTRUCTION,08-Jan-14,268 Madison St,N200711077587
151,2.0,1.0,0,0,"(37.725757999999999, -122.42266100000001)",5943008,CONSTRUCTION,08-Jan-14,268 Madison St,N200711077587
758,1.0,1.0,0,0,"(37.714331000000001, -122.404082)",6208056,BP ISSUED,25-Apr-13,138 Alpha St,N200806194898
159,1.0,1.0,0,0,"(37.714095, -122.404408)",6208003,BP ISSUED,25-Apr-13,138 Alpha St,N200806194898



Duplicated records at 2014'Q2


...

,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE,LOCATION,BP_APPLNO
794,2,2,0,0,1078 HAMPSHIRE ST,4152046,CONSTRUCTION,02/21/2012,"(37.75487194240, -122.40775958600)",N200709193092
863,2,1,0,0,1076 HAMPSHIRE ST,4152016,CONSTRUCTION,02/21/2012,"(37.75494040740, -122.40776607900)",N200709193092
523,2,2,0,0,443 LINDEN ST,818049,CONSTRUCTION,10/10/2014,"(37.77599145770, -122.42523007200)",N200912304034
782,2,2,0,0,447 LINDEN ST,818048,CONSTRUCTION,10/10/2014,"(37.77596050200, -122.42532017400)",N200912304034



Duplicated records at 2015'Q3


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE,LOCATION,BP_APPLNO
321,8,6,0,0,95 LELAND AV,6250028,BP REINSTATED,03/05/2010,"94134\n(37.71168292210, -122.40561714300)",N200704128664
346,8,5,0,0,95 LELAND AV,6250037,BP REINSTATED,03/05/2010,"94134\n(37.71172071370, -122.40571413900)",N200704128664
170,59,59,7,7,249 PENNSYLVANIA AV,3999002,BP Filed,05/29/2015,"94107\n(37.76297137300, -122.39317202300)",N201505297549
222,16,16,0,0,502 07TH ST,3780001,BP FILED,05/29/2015,"94103\n(37.77399389340, -122.40478807300)",N201505297549



Duplicated records at 2016'Q2


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE,LOCATION,BP_APPLNO
415,2,1,0,0,1948 & 1948A QUESADA AV,5329011,BP FILED,04/11/2016,"(37.73625912340, -122.39606265400)",N201604114413
416,1,1,0,0,1948 QUESADA AV,5329011,BP FILED,04/11/2016,"(37.73625912340, -122.39606265400)",N201604114413


# Fixing duplicates
Re-adding duplicate records with corrected values.

In [6]:
def fix_duplicate(permitId, units, netUnits, affUnits, netAffUnits, addr, apn, beststat, bestdate):
    for i in range(count):
        if len(duplicates[i][duplicates[i]['BP_APPLNO'] == permitId]) > 0:
            data[i].loc[permitId] = [units, netUnits, affUnits, netAffUnits, addr, apn, beststat, bestdate]
        duplicates[i] = duplicates[i][duplicates[i]['BP_APPLNO'] != permitId]

fix_duplicate('N200709193092', 2, 1, 0, 0, '1076-1078 Hampshire St', '4152016', "CONSTRUCTION", date(2012, 2, 21))
fix_duplicate('N200711077587', 2, 1, 0, 0, '268 Madison St', '5943008', "CONSTRUCTION", date(2014, 2, 8))
fix_duplicate('N200707055953', 7, 7, 0, 0, '4132 03rd St', '5260003', "CONSTRUCTION", date(2014, 7, 28))
fix_duplicate('N200912304034', 2, 2, 0, 0, '447 Linden St', '818048', "CONSTRUCTION", date(2014, 10, 10))
fix_duplicate('N201209059006', 191, 191, 0, 0, '218 Buchanan St', '857001', "CONSTRUCTION", date(2014, 7, 31))
fix_duplicate('N200711077576', 1, 1, 0, 0, '83 Panorama Dr', '2821010', "CONSTRUCTION", date(2014, 9, 23))
fix_duplicate('N201207104447', 98, 98, 0, 0, '1100 GOLDEN GATE AV', '757025', "CONSTRUCTION", date(2015, 3, 27))
fix_duplicate('N201305036062', 139, 139, 0, 0, '555 FULTON ST', '794028', "CONSTRUCTION", date(2015, 3, 31))

hasDuplicates = False
for i in range(count):
    if (len(duplicates[i]) > 0):
        print()
        print("Duplicated records at {}".format(labels[i]))
        display(duplicates[i])
        hasDuplicates = True
if not hasDuplicates:
    print("No duplicates present!")

ValueError: Must have equal len keys and value when setting with an iterable

# Displaying records without Permit ID
Printing all records without Permit ID

In [7]:
for i in range(count):
    if (len(missing[i]) > 0):
        print()
        print("Records without Permit ID at {}".format(labels[i]))
        display(missing[i])


Records without Permit ID at 2013'Q4


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE,LOCATION,BP_APPLNO
2,165.0,165.0,0,0,"(37.775286399999999, -122.4159598)",3507042,PL FILED,08-Apr-09,1400 Mission St,N
3,7.0,4.0,0,0,"(37.785198800000003, -122.41015710000001)",331001A,PL Filed,28-Feb-13,231 Ellis St,N
5,170.0,170.0,0,0,"(37.795891599999997, -122.4005452)",201012,PL Approved,19-Jun-12,8 Washington Street,N
7,6.0,6.0,0,0,"(37.797133100000003, -122.4139845)",150054,PL Filed,16-Jan-13,1020 Broadway,N
9,28.0,28.0,0,0,"(37.792779000000003, -122.419763)",620006,PL Approved,15-Nov-13,1601 Larkin St,N
17,3.0,2.0,0,0,"(37.801653100000003, -122.4058114)",105065,PL Filed,21-Aug-13,377 Filbert St,N
21,170.0,170.0,0,0,"(37.7842451, -122.4103786)",331010,PL Approved,26-Mar-09,168 Eddy St,N
28,45.0,45.0,0,0,"(37.789756300000001, -122.4050955)",287013,PL Approved,06-Oct-11,300 Grant Ave.,N
36,26.0,26.0,0,0,"(37.799661999999998, -122.424617)",522002A,PL Filed,28-Aug-13,2601 Van Ness Av,N
39,2.0,1.0,0,0,"(37.798259999999999, -122.430943)",531004,PL Approved,22-Oct-09,2919 Laguna St,N



Records without Permit ID at 2014'Q1


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE,LOCATION,BP_APPLNO
4,2.0,1.0,0,0,"(37.780580299999997, -122.49460500000001)",1466009,PL Approved,16-Jul-09,451 34th Av,N
11,8619.0,7800.0,0,0,"(37.823551500000001, -122.370648)",1939001,PL Approved,15-Mar-11,Treasure Island,N
26,8.0,8.0,0,0,"(37.767417000000002, -122.420868)",3547032,PL Approved,26-Jan-12,49 Julian Av,N
48,185.0,185.0,0,0,"(37.786009900000003, -122.40253869999999)",3706093,PL Approved,31-Jul-13,706 Mission St,N
56,39.0,39.0,0,0,"(37.778765999999997, -122.41091900000001)",3726103,PL Filed,14-Aug-12,114 07th Street,N
59,119.0,119.0,0,0,"(37.7877662, -122.3929342)",3747012,PL Filed,02-May-13,325 Fremont St,N
62,1700.0,1094.0,0,0,"(37.757050900000003, -122.39567030000001)",4167004,PL Filed,30-Jun-10,1 Turner Tr,N
66,42.0,42.0,0,0,"(37.775635000000001, -122.409447)",3730023,PL Filed,29-Jul-13,1174 Folsom St,N
76,398.0,398.0,0,0,"(37.787269000000002, -122.39631)",3736074,PL Approved,14-Nov-13,41 Tehama St,N
83,13.0,69.0,0,0,"(37.779603999999999, -122.404006)",3753140,PL Filed,19-Apr-10,935 Folsom St,N



Records without Permit ID at 2014'Q2


...

,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE,LOCATION,BP_APPLNO
0,4,4,0,0,3140 16TH ST,3555018,PL APPROVED,07/27/2016,"(37.76510622520, -122.42289112700)",N
2,15,15,0,0,824 HYDE ST,0280017,PL APPROVED,03/24/2016,"(37.78857856960, -122.41659181800)",N
31,96,96,96,96,1296 SHOTWELL ST,6571051,PL FILED,04/04/2016,"(37.74864031080, -122.41520050200)",N
43,220,220,0,0,1601 MISSION ST,3514043,PL APPROVED,04/07/2016,"(37.77204073130, -122.41876324900)",N
52,4,3,0,0,21 ROSEMONT PL,3534020,PL APPROVED,07/23/2014,"(37.76876056660, -122.42540361800)",N
71,37,37,0,0,1245 FOLSOM ST,3756041,PL FILED,03/22/2016,"(37.77418218890, -122.41048159100)",N
84,359,359,0,0,1270 MISSION ST,3701021,PL FILED,06/28/2016,"(37.77680707790, -122.41452278800)",N
91,172,172,28,28,1301 16TH ST,3954016,PL APPROVED,06/28/2016,"(37.76584514550, -122.40023935500)",N
103,2,1,0,0,132 CORBETT AV,2652010,PL APPROVED,07/10/2014,"(37.76186080670, -122.44127466600)",N
104,124,18,0,0,1320 - 1380 LOMBARD ST,0499002D,PL FILED,01/12/2016,"(37.80176299930, -122.42372989200)",N



Records without Permit ID at 2016'Q4


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE,LOCATION,BP_APPLNO
1,94,94,0,0,955 POST ST,0302021,PL FILED,12/09/2016,"(37.78683277040, -122.41763739600)",N
5,164,164,0,0,1125 MARKET ST,3702047,PL FILED,12/18/2013,"(37.77975627180, -122.41273619900)",N
61,37,37,0,0,1245 FOLSOM ST,3756041,PL FILED,03/22/2016,"(37.77418218890, -122.41048159100)",N
76,94,94,94,94,1296 SHOTWELL ST,6571051,PL APPROVED,12/07/2016,"Y\n(37.74864031080, -122.41520050200)",N
84,172,172,28,28,1301 16TH ST,3954016,PL APPROVED,06/28/2016,"(37.76584514550, -122.40023935500)",N
86,231,231,0,0,1333 GOUGH ST / 1481 POST ST,0697037,PL FILED,11/13/2013,"(37.78573380920, -122.42560544400)",N
117,17,2,0,0,140 DUBOCE AV,3502024,PL FILED,09/11/2014,"Y\n(37.77001322180, -122.42349678000)",N
129,63,63,8,8,1436 & 1498 POLK ST/ 1567 CALIFORNIA ST,0645014,PL APPROVED,10/16/2015,"(37.79026564810, -122.42029936200)",N
134,4,2,0,0,146 JASPER PL,0103021,PL FILED,06/24/2013,"(37.80125558020, -122.40879053700)",N
138,45,45,0,0,1463 STEVENSON ST,3532013,PL FILED,06/28/2016,"Y\n(37.76845891620, -122.42109368400)",N



Records without Permit ID at 2017'Q1


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE,LOCATION,BP_APPLNO
7,164,164,0,0,1125 MARKET ST,3702047,PL FILED,12/18/2013,"(37.77975627180, -122.41273619800)",N
58,37,37,0,0,1245 FOLSOM ST,3756041,PL FILED,03/22/2016,"(37.77418218980, -122.41048158900)",N
78,94,94,94,94,1296 SHOTWELL ST,6571051,PL APPROVED,12/07/2016,"Y\n(37.74864030990, -122.41520050200)",N
98,44,44,0,0,230 07TH ST,3730004,PL FILED,07/30/2014,"(37.77729530030, -122.40912349300)",N
104,18,16,0,0,231 ELLIS ST,0331001A,PL FILED,02/28/2013,"(37.78501273230, -122.40994765900)",N
113,4,3,0,0,3532 23RD ST,3634009,PL APPROVED,07/08/2016,"Y\n(37.75385569910, -122.42234910200)",N
126,172,172,28,28,1301 16TH ST,3954016,PL APPROVED,06/28/2016,"(37.76584514460, -122.40023935500)",N
137,124,18,0,0,1320 - 1380 LOMBARD ST,0499002D,PL FILED,01/12/2016,"(37.80176300020, -122.42372989200)",N
138,4,1,0,0,1331 WASHINGTON ST,0215023,PL FILED,06/30/2016,"(37.79386950560, -122.41530254700)",N
139,231,231,0,0,1333 GOUGH ST / 1481 POST ST,0697037,PL FILED,11/13/2013,"(37.78573381010, -122.42560544300)",N


# Fixing records without Permit ID
Correcting records without Permit ID

In [8]:
def fix_missing(permitId, units, netUnits, affUnits, netAffUnits, addr, apn):
    for i in range(count):
        if len(missing[i][missing[i]['APN'].isin(apn)]) > 0:
            data[i].loc[permitId] = [units, netUnits, affUnits, netAffUnits, addr, apn[0]]
        missing[i] = missing[i][~missing[i]['APN'].isin(apn)]
        
fix_missing("N_CUSTOM_JAMESTOWN", 132, 14, 0, 0, '833-881 Jamestown', ['4991277'])
fix_missing("N_CUSTOM_BROTHERHOOD", 196, 196, 0, 0, '800 BROTHERHOOD WAY', ['7331003', '7331005'])
fix_missing("N_CUSTOM_SHIPYARD", 229, 229, 0, 0, 'HUNTERS POINT SHIPYARD, PHASE I', ['4591C001', '4624031'])

hasMissing = False
for i in range(count):
    if (len(missing[i]) > 0):
        print()
        print("Records without Permit ID at {}".format(labels[i]))
        display(missing[i])
        hasMissing = True
if not hasMissing:
    print("No missing present")

ValueError: cannot set a row with mismatched columns

# Searching for incorrect unit values
Searching projects with unit counts varying from quarter to quarter and projects with net units exceeding total units.

In [ ]:
def find_incorrect(column):
    incorrect = {}
    for i in range(count-1):
        X = data[i]
        for j in range(i+1, count):
            X2 = data[j]
        
            X = X[X.index.isin(X2.index)].sort_index()
            X2 = X2[X2.index.isin(X.index)].sort_index()
            Y = X[X[column] != X2[column]]
            Y2 = X2[X[column] != X2[column]]
            if len(Y)>0:
                for index, row in Y.iterrows():
                    a = int(Y.loc[index][column])
                    b = int(Y2.loc[index][column])
                    if index not in incorrect:
                        incorrect[index] = { 'min': min(a, b), 'max': max(a,b)}
                    else:
                        incorrect[index] = { 'min': min(incorrect[index]['min'], a, b), 'max': max(incorrect[index]['max'], a, b)}
    return incorrect


incorrect = find_incorrect('UNITS')
print("Unit number fluctuation")
display(incorrect)

incorrectNet = find_incorrect('NET_UNITS')
print("Net Unit number fluctuation")
display(incorrectNet)

In [ ]:
def fix_units(permitId, units, column):
    for i in range(count):
        if permitId in data[i].index:
            d = data[i].loc[permitId]
            d[column] = units
            data[i].loc[permitId] = d

for k in incorrect:
    fix_units(k, incorrect[k]['max'], 'UNITS')
for k in incorrectNet:
    fix_units(k, incorrectNet[k]['max'], 'NET_UNITS')    
    
# 201 Folsom
fix_units('N201207124717', 656, 'UNITS')
fix_units('N201207124717', 656, 'NET_UNITS')

# Detection of completed buildings, p.1
Identifying completed projects as the ones that were in construction in a given quarter and are removed from the pipeline in the following quarter.

In [9]:
def buildStats(data):
    buildings = list(map(lambda x: len(x), completed))
    units = list(map(lambda x: x['UNITS'].values.sum(), completed))
    netUnits = list(map(lambda x: x['NET_UNITS'].values.sum(), completed))
    aff = list(map(lambda x: x['AFF_UNITS'].values.sum(), completed[4:]))
    netAff = list(map(lambda x: x['NET_AFF_UNITS'].values.sum(), completed[4:]))
    return { "buildings": buildings, "units": units, "netUnits": netUnits, "aff": aff, "netAff": netAff }

def printStats(stats):
    print("Buildings: {}".format(stats["buildings"]))
    print("Net Units: {}".format(stats["netUnits"]))
    print("Total Units: {}".format(sum(stats["netUnits"])))
    print("Net Affordable Units: {}".format(stats["netAff"]))
    print("Total Affordable Units: {}".format(sum(stats["netAff"])))

def contains(data, key):
    for i in range(len(data)):
        if key in data[i].index:
            return True
    return False

In [10]:
completed = []
for i in range(len(data)-1):
    X = data[i]
    X2 = data[i+1]
    X = X[~X.index.isin(X2.index)]
    completed.append(X)

stats = buildStats(completed)
printStats(stats)

Buildings: [28, 37, 32, 112, 37, 43, 29, 40, 110, 103, 45, 107, 87]
Net Units: [1889, 628, 736, 2981, 909, 1114, 615, 911, 1873, 1418, 964, 3262, 1161]
Total Units: 18461
Net Affordable Units: [161, 10, 34, 167, 126, 101, 175, 116, 75]
Total Affordable Units: 965


# Detection of completed buildings, p.2
Correcting for projects that re-appear in the pipeline after being removed.

In [59]:
completed = []
for i in range(len(data)-1):
    X = data[i]
    for j in range(i+1,len(data)):
        X2 = data[j]
        X = X[~X.index.isin(X2.index)]
    completed.append(X)

stats = buildStats(completed)
printStats(stats)

Buildings: [27, 36, 32, 75, 31, 34, 28, 39, 110, 103, 44, 104, 87]
Net Units: [1879, 627, 736, 2006, 744, 578, 855, 931, 2118, 1819, 1171, 3767, 1247]
Total Units: 18478
Net Affordable Units: [1, 10, 34, 167, 126, 101, 175, 23, 75]
Total Affordable Units: 712


# Percent of units in top 10 projects for each quartal
Here we print the top ten projects (by net new units) for each quarter and compare their contribution to the total net number of units completed in the same period. As we see below, in a typical quarter the top ten projects account for 90%+ of citywide housing production.

In [60]:
for i in range(len(labels)-1):
    topUnits = completed[i].sort_values('NET_UNITS', ascending=False).head(10)['NET_UNITS'].values.sum()
    units = completed[i]['NET_UNITS'].values.sum()
    print(labels[i+1])
    print(topUnits/units)

2014'Q1
0.973922299095
2014'Q2
0.912280701754
2014'Q3
0.953804347826
2014'Q4
0.847956131605
2015'Q1
0.961021505376
2015'Q2
0.956747404844
2015'Q3
0.983625730994
2015'Q4
0.924812030075
2016'Q1
0.847025495751
2016'Q2
0.911489829577
2016'Q3
0.949615713066
2016'Q4
0.701088399257
2017'Q1
0.850040096231


# Group Stats by Year

In [61]:
for i in range(3):
    yearCount = int(sum(stats['netUnits'][i*4:i*4+4]))
    print("Year {}:      {}".format(2014 + i, yearCount))
print("Year 2017 (Q1): {}".format(completed[len(completed) - 1]['NET_UNITS'].sum()))

Year 2014:      5248
Year 2015:      3108
Year 2016:      8875
Year 2017 (Q1): 1247


# Result Table (top 10)

In [62]:
for i in range(len(labels)-1):
    print(labels[i+1])
    display(completed[i].sort_values('NET_UNITS', ascending=False).head(10))

2014'Q1


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE
PERMIT_ID,,,,,,,,
N200607207084,754,754,0,0,1401 Market St,3507041,CONSTRUCTION,2012-02-23
N201104224606,315,315,0,0,185 Channel St,8711023,CONSTRUCTION,2013-12-13
N201207054130,273,273,0,0,1155 04th St,8713001,CONSTRUCTION,2013-12-23
N201203135987,172,172,0,0,1200 04th St,8711017,BP ISSUED,2012-03-26
N201012217106,115,115,0,0,1960-1998 Market St,872005,CONSTRUCTION,2013-12-19
N200506246051,88,88,0,0,333 Fremont St,3747019,CONSTRUCTION,2013-12-12
N200912223711,52,52,0,0,63 West Point Rd,4624004,CONSTRUCTION,2013-04-09
N201109074027,24,24,0,0,1600 Market St,854001,CONSTRUCTION,2013-12-30
N200701051074,19,19,0,0,246 Ritch St,3776092,CONSTRUCTION,2013-10-23


2014'Q2


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE
PERMIT_ID,,,,,,,,
N201106017202,182,182,0,0,260 05th St,3732008,CONSTRUCTION,2014-03-31
N200210249843,134,134,0,0,555 Market St,3708058,PL APPROVED,2002-12-05
N201111038205,40,40,0,0,1501 15th St,3553054,CONSTRUCTION,2014-02-25
N201110146841,38,38,0,0,1645-1661 Pacific Av,595013,CONSTRUCTION,2014-03-24
N201207164905,36,36,0,0,345 06th St,3753081,BP Filed,2012-07-16
N201012227225,35,35,0,0,495 Cambridge St,5992A060,BP Filed,2010-12-22
N200608290880,35,35,0,0,1080 Sutter St,279011,CONSTRUCTION,2014-01-14
N200602104391,29,29,0,0,793 South Van Ness Av,3591024,PL Filed,2012-08-14
N201006295585,23,23,0,0,42 Harriet St,3731101,PL Approved,2010-12-20


2014'Q3


...

,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE
PERMIT_ID,,,,,,,,
N201209059006,450,450,23,23,55 LAGUNA ST (BLDG 2),0857001,CONSTRUCTION,2016-04-29
N201312184508,260,260,31,31,1634 - 1690 PINE ST,0647007,CONSTRUCTION,2016-07-01
N201207104447,98,98,98,98,1100 GOLDEN GATE AV,0757025,CONSTRUCTION,2016-07-01
N201306250465,84,84,11,11,480 POTRERO AV,3973002C,CONSTRUCTION,2016-01-13
N201207124725,80,80,0,0,LUMINA (PLAZA C),3746001,CONSTRUCTION,2016-07-01
N201306260573,41,41,5,5,450 HAYES ST,0808039,CONSTRUCTION,2016-06-29
N200912032516,35,35,5,5,1 FRANKLIN ST,0837003,CONSTRUCTION,2016-06-16
N201406138386,27,27,2,2,832 SUTTER ST,0281003,CONSTRUCTION,2016-01-21
N200507208144,25,25,0,0,468 CLEMENTINA ST,3732071,CONSTRUCTION,2015-10-01


2016'Q4


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE
PERMIT_ID,,,,,,,,
N201306250394,560,560,0,0,1 HENRY ADAMS ST,3911001,CONSTRUCTION,2016-04-07
N201112070227,470,470,0,0,POTRERO1010 (BLDG 1 OF 2),3833001,CONSTRUCTION,2016-05-26
N201511092111,313,313,0,0,55 CHUMASERO DR,7330001,BP FILED,2015-11-09
N201510260810,299,299,0,0,1208 JUNIPERO SERRA BL,7326001,BP FILED,2015-10-26
N201510230640,248,248,0,0,455 SERRANO DR,7335001,BP FILED,2015-10-23
N201409116118,202,202,0,0,1095 MARKET ST,3703059,CONSTRUCTION,2016-07-28
N201609208248,174,174,0,0,5 THOMAS MELLON CIRCLE,4991075,BP FILED,2016-09-20
N201609208258,133,133,0,0,5 THOMAS MELLON CIRCLE,4991075,BP FILED,2016-09-20
N201312265046,129,129,0,0,360 BERRY ST,8704004,CONSTRUCTION,2016-08-22


2017'Q1


,UNITS,NET_UNITS,AFF_UNITS,NET_AFF_UNITS,NAMEADDR,APN,BESTSTAT,BESTDATE
PERMIT_ID,,,,,,,,
N201307303137,416,416,62,62,350 08TH ST,3756003,CONSTRUCTION,2016-11-03
N201404042522,200,200,0,0,MISSION BAY BLOCK 7,8711031,CONSTRUCTION,2016-12-08
N201507060668,79,79,0,0,55 LAGUNA ST (SENIOR CENTER),0857002,BP ISSUED,2016-05-27
N200711077587,80,77,0,0,268 MADISON ST,5943008,CONSTRUCTION,2014-01-08
N201311222660,77,77,0,0,2101 & 2155 WEBSTER ST,0629037,CONSTRUCTION,2016-11-15
N200809252660,69,69,9,9,1450 FRANKLIN ST,0671006,CONSTRUCTION,2016-12-30
N201306270646,40,40,0,0,55 LAGUNA ST (AFF SENIOR HOUSING),0857002,CONSTRUCTION,2016-11-10
N201412294591,38,38,4,4,980 FOLSOM ST,3732028,BP FILED,2014-12-29
N200806164548,76,37,0,0,226 06TH ST,3731003,CONSTRUCTION,2014-10-20


# Exporting datasets

In [18]:
for i in range(count):
    fname = 'data_complete/PipelineCleaned_'+labels[i].replace('\'','_')+'.csv'
    data[i].to_csv(fname)
for i in range(count-1):
    fname = 'data_complete/PipelineCompleted_'+labels[i+1].replace('\'','_')+'.csv'
    completed[i].to_csv(fname)

# Total in pipeline per quartal

In [49]:
for i in range(count):
    print("{}: {}".format(labels[i], data[i]['NET_UNITS'].values.sum()))

2013'Q4: 6410
2014'Q1: 5106
2014'Q2: 6187
2014'Q3: 7246
2014'Q4: 7793
2015'Q1: 7709
2015'Q2: 8651
2015'Q3: 8990
2015'Q4: 8717
2016'Q1: 8291
2016'Q2: 7960
2016'Q3: 7173
2016'Q4: 6329
2017'Q1: 5680
